In [1]:
import pandas as pd
import numpy as np

In [ ]:
# rate=prescr_merge[['County','Year','Prescr_rate']].copy()
# #rate['bystaterate']=prescr_merge[['County','Year','Prescr_rate']].groupby(['Year']).sum().reset_index()['Prescr_rate']
# av=prescr_merge[['County','Year','Prescr_rate']].groupby(['Year']).mean().reset_index()
# #handle missing data for Glades.The only county missing

# #glades=prescr_merge[prescr_merge.County=='Glades']
# all=pd.merge(av,prescr_merge,on=['Year'],how='inner')
# def deaths(row):
#     if np.isnan(row['MME']):
#         row['MME'] = row['Population'] * row['Prescr_rate_x']

#     return row['MME']

# all['MME'] = all.apply(deaths, axis=1)
# all['Prescr_rate_y']=all['MME']/all['Population']
# all['Prescr_rate_y']=all['MME']/all['Population']
# Florida_updated=all[['State','Year','County','Prescr_rate_y','Treatment']]
# Florida_updated.head()
# Florida_updated=all[['State','Year','County','Prescr_rate_y','Treatment']]
# Florida_updated.head()

In [11]:
# get all data
deaths=pd.read_csv('deaths.csv')
populations=pd.read_csv('CountyPopulations.csv')
prescriptions=pd.read_csv('prescriptions')

In [12]:
prescriptions

,Unnamed: 0,BUYER_COUNTY,T_DATE,MME,State
0,0,ALACHUA,2006,1.645948e+12,Florida
1,1,ALACHUA,2007,1.972812e+12,Florida
2,2,ALACHUA,2008,2.540522e+12,Florida
3,3,ALACHUA,2009,3.180395e+12,Florida
4,4,ALACHUA,2010,3.293852e+12,Florida
...,...,...,...,...,...
3917,1066,WOODFORD,2010,1.128719e+10,Kentucky
3918,1067,WOODFORD,2011,1.402864e+10,Kentucky
3919,1068,WOODFORD,2012,1.461070e+10,Kentucky
3920,1069,WOODFORD,2013,1.364608e+10,Kentucky


## FLORIDA

In [13]:
prescriptions['County']=prescriptions['BUYER_COUNTY']
florida=prescriptions[prescriptions.State=='Florida'][['County','T_DATE','MME','State']]
flcounties=prescriptions[prescriptions.State=='Florida']['County'].unique()
missing=[]
for j in range(2006,2015):
    for i in flcounties:
        if i not in[l for l in florida[(florida.County==i)&(florida.T_DATE==j)]['County']]:
            missing.append([i,j,'NA','Florida']) # giving NA for the counties with no prescription related info
missing_dataframe=pd.DataFrame(missing,columns=['County','T_DATE','MME','State'])
updated_prescr=pd.concat([florida,missing_dataframe],axis=0)
updated_prescr['Treatment']='Treatment'
updated_prescr.groupby(['T_DATE'])['County'].count() #double checking that all florida counties are included for each year

T_DATE
2006    68
2007    68
2008    68
2009    68
2010    68
2011    68
2012    68
2013    68
2014    68
Name: County, dtype: int64

## Georgia

In [14]:
Georgia=prescriptions[prescriptions.State=='Georgia'][['County','T_DATE','MME','State']].copy()
Georgia_counties=prescriptions[(prescriptions.State=='Georgia')&(prescriptions.T_DATE==2013)]['BUYER_COUNTY'].unique()
Georgiafinal=Georgia.loc[Georgia.County.isin(Georgia_counties),].copy()
Georgiafinal['Treatment']='Control'
Georgia.loc[Georgia.County.isin(Georgia_counties),].groupby('T_DATE')['County'].count() #double checking that all georgia counties are included for each year

T_DATE
2006    151
2007    151
2008    151
2009    151
2010    151
2011    151
2012    151
2013    151
2014    151
Name: County, dtype: int64

## North Carolina

In [15]:
NC=prescriptions[prescriptions.State=='North Carolina'][['County','T_DATE','MME','State']]
NC_counties=prescriptions[(prescriptions.State=='North Carolina')&(prescriptions.T_DATE==2013)]['BUYER_COUNTY'].unique()
NCfinal=NC.loc[NC.County.isin(NC_counties),]
NCfinal['Treatment']='Control'
NC.loc[NC.County.isin(NC_counties),].groupby('T_DATE')['County'].count() # confirm all years have the same counties

T_DATE
2006    98
2007    98
2008    98
2009    98
2010    98
2011    98
2012    98
2013    98
2014    98
Name: County, dtype: int64

## Kentucky

In [16]:
KY=prescriptions[prescriptions.State=='Kentucky'][['County','T_DATE','MME','State']]
KY_counties=prescriptions[(prescriptions.State=='Kentucky')&(prescriptions.T_DATE==2013)]['BUYER_COUNTY'].unique()
KYfinal=KY.loc[KY.County.isin(KY_counties),]
KYfinal['Treatment']='Control'
KY.loc[KY.County.isin(KY_counties),].groupby('T_DATE')['County'].count() # confirm all years have the same counties

T_DATE
2006    119
2007    119
2008    119
2009    119
2010    119
2011    119
2012    119
2013    119
2014    119
Name: County, dtype: int64

## Putting all States together+ population

In [17]:
all_prescriptions=pd.concat([updated_prescr,KYfinal,Georgiafinal,NCfinal],axis=0)
all_prescriptions.head()

,County,T_DATE,MME,State,Treatment
0,ALACHUA,2006,1645948320669.000732,Florida,Treatment
1,ALACHUA,2007,1972812101019.285645,Florida,Treatment
2,ALACHUA,2008,2540521825968.202148,Florida,Treatment
3,ALACHUA,2009,3180395323310.341309,Florida,Treatment
4,ALACHUA,2010,3293852490635.906738,Florida,Treatment


In [18]:
states=['Florida', 'Georgia', 'North Carolina', 'Kentucky']
subset=populations.loc[populations.State.isin(states),].copy() #only getting population info on those states
subset['County']=[i.split(' County')[0] for i in subset.County]
all_prescriptions['County']=(all_prescriptions['County'].str.lower()).str.capitalize()
all_prescriptions['Year']=all_prescriptions['T_DATE'] #renaming the column

In [19]:
rectifying = {
    "De soto": "DeSoto",
    "Indian river": "Indian River",
    "Miami-dade": "Miami-Dade",
    "Palm beach": "Palm Beach",
    "Saint johns": "St. Johns",
    "Saint lucie": "St. Lucie",
    "Saint Johns": "St. Johns",
    "Saint Lucie": "St. Lucie",
    "Santa rosa": "Santa Rosa",
    "Mccracken": "McCracken",
    "Mccreary": "McCreary",
    "Mclean": "McLean",
    "Ben hill": "Ben Hill",
    "Dekalb": "DeKalb",
    "Jeff davis": "Jeff Davis",
    "Mcduffie": "McDuffie",
    "Mcintosh": "McIntosh",
    "Mcdowell": "McDowell",
    "New hanover": "New Hanover",
}
p=[]
for i in all_prescriptions['County']:
    if i in rectifying.keys():
        p.append(rectifying[i])
    else:
        p.append(i)
all_prescriptions['County']=p

# Dealing with NA data

In [20]:
prescr_merge=pd.merge(all_prescriptions,subset,on=['County','Year','State'],how='left',indicator=True)[['State','County','MME','Population','Year','Treatment','_merge']]
prescr_merge[prescr_merge["_merge"] != "both"]

,State,County,MME,Population,Year,Treatment,_merge
564,Florida,Unknown,24518.7,NaN,2006,Treatment,left_only
565,Florida,Unknown,14832.3,NaN,2007,Treatment,left_only
605,Florida,Unknown,NA,NaN,2008,Treatment,left_only
606,Florida,Unknown,NA,NaN,2009,Treatment,left_only
607,Florida,Unknown,NA,NaN,2010,Treatment,left_only
608,Florida,Unknown,NA,NaN,2011,Treatment,left_only
609,Florida,Unknown,NA,NaN,2012,Treatment,left_only
610,Florida,Unknown,NA,NaN,2013,Treatment,left_only
611,Florida,Unknown,NA,NaN,2014,Treatment,left_only


In [21]:
prescr_merge['MME']=prescr_merge['MME'].replace('NA',np.NaN) 
#prescr_merge['MME']=[np.random.randint(0,10) if nfor i in prescr_merge['MME']] #should we just remove this row
prescr_merge['Prescr_rate']=prescr_merge['MME']/prescr_merge['Population'] 

In [22]:
prescr_merge[prescr_merge.isnull().any(axis=1)] #will need to talk about how to deal with these 2 Counties. Not a big deal!

,State,County,MME,Population,Year,Treatment,_merge,Prescr_rate
564,Florida,Unknown,24518.7,NaN,2006,Treatment,left_only,NaN
565,Florida,Unknown,14832.3,NaN,2007,Treatment,left_only,NaN
602,Florida,Glades,NaN,11093.0,2006,Treatment,both,NaN
603,Florida,Glades,NaN,10967.0,2007,Treatment,both,NaN
604,Florida,Glades,NaN,11072.0,2008,Treatment,both,NaN
605,Florida,Unknown,NaN,NaN,2008,Treatment,left_only,NaN
606,Florida,Unknown,NaN,NaN,2009,Treatment,left_only,NaN
607,Florida,Unknown,NaN,NaN,2010,Treatment,left_only,NaN
608,Florida,Unknown,NaN,NaN,2011,Treatment,left_only,NaN
609,Florida,Unknown,NaN,NaN,2012,Treatment,left_only,NaN


In [23]:
prescr_pop=prescr_merge.to_csv("Prescription-Pop-merge.csv",index=False)